In [ ]:
import tvm

import tvm.te as te
import tvm.relax as rx
import tvm.tir as tir

from tvm.script import ir as I
from tvm.script import tir as T
from tvm.script import relax as R

from tvm.relax.binding_rewrite import DataflowBlockRewrite
from tvm.relax.analysis import name_to_binding


def showmod(mod: tvm.ir.module.IRModule):
    mod.show(
        black_format=True,
        show_meta=False,
        verbose_expr=True,
        show_object_address=False,
        show_all_struct_info=True,
    )


def createandshowmod(ops):
    te_func = te.create_prim_func(ops).with_attrs({"global_symbol": "test"})
    mod = tvm.IRModule({"test": te_func})
    showmod(mod)

In [ ]:
testSuites = {}


def register(func):
    if func.__name__ in testSuites:
        raise Exception("Duplicated test suite name: " + func.__name__)
    else:
        testSuites[func.__name__] = func
    return func

In [ ]:
@register
def test_block_builder():
    m = tir.Var("m", "int64")
    n = tir.Var("n", "int64")

    x = rx.Var("x", rx.TensorStructInfo([m, n], "float32"))
    y = rx.Var("y", rx.TensorStructInfo([m, n], "float32"))

    bb = rx.BlockBuilder()

    with bb.function("func", [x, y]):
        with bb.dataflow():
            lv0 = bb.emit(rx.op.add(x, y))
            assert lv0.name_hint == "lv"
            gv0 = bb.emit_output(lv0)

        bb.emit_func_output(gv0)

    # Finalize the building process and return the result IRModule.
    mod = bb.finalize()
    func = mod["func"]

    showmod(mod)
    showmod(func)


testSuites["test_block_builder"]()